# Домашнее задание № 4 часть 2 (курс МТС, команда 33)

## Подготовка окружения для проведения эксперимента

In [1]:
# устанавливаем необходимые зависимости
!pip install rectools > None

In [2]:
# импортируем зависимости

import pandas as pd
import numpy as np

import nmslib
from rectools.metrics import Recall, MAP, calc_metrics
from rectools.models import LightFMWrapperModel
from rectools import Columns
from rectools.dataset import Dataset
from lightfm import LightFM
from lightfm.data import Dataset as LFMDataset

Your CPU supports instructions that this binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2
For maximum performance, you can install NMSLIB from sources 
pip install --no-binary :all: nmslib


In [3]:
# скачиваем датасеты

!wget https://storage.yandexcloud.net/itmo-recsys-public-data/kion_train.zip

--2022-12-11 20:36:45--  https://storage.yandexcloud.net/itmo-recsys-public-data/kion_train.zip
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 78795385 (75M) [application/zip]
Saving to: ‘kion_train.zip’

kion_train.zip      100%[===================>]  75.14M  22.0MB/s    in 4.2s    

2022-12-11 20:36:50 (17.8 MB/s) - ‘kion_train.zip’ saved [78795385/78795385]



In [4]:
!ls

None  __notebook__.ipynb  kion_train.zip


In [5]:
!unzip ./kion_train.zip 

Archive:  ./kion_train.zip
   creating: kion_train/
  inflating: kion_train/interactions.csv  
  inflating: __MACOSX/kion_train/._interactions.csv  
  inflating: kion_train/users.csv    
  inflating: __MACOSX/kion_train/._users.csv  
  inflating: kion_train/items.csv    
  inflating: __MACOSX/kion_train/._items.csv  


In [6]:
N = 10
METRICS = {"map@10": MAP(k=10),"recall@10": Recall(k=10)}

In [7]:
def get_datasets():
    inter_df = pd.read_csv('kion_train/interactions.csv')
    user_df = pd.read_csv('kion_train/users.csv')
    item_df = pd.read_csv('kion_train/items.csv')
    inter_df.rename(columns={'last_watch_dt': 'datetime','total_dur': 'weight',},inplace=True)
    inter_df['datetime'] = pd.to_datetime(inter_df['datetime'])
    inter_df[Columns.Weight] = np.where(inter_df['watched_pct'] > 10, 3, 1)
    return inter_df, user_df, item_df

In [8]:
inter_df, user_df, item_df = get_datasets()

# формируем трейновый и тестовый датасет

last_date = inter_df[Columns.Datetime].max().normalize()

train = inter_df[inter_df[Columns.Datetime] < last_date - pd.Timedelta(days=7)].copy()
test = inter_df[inter_df[Columns.Datetime] >= last_date - pd.Timedelta(days=7)].copy()

print(f"Размерность трейна : {train.shape}")
print(f"Размерность теста: {test.shape}")

Размерность трейна : (4985269, 5)
Размерность теста: (490982, 5)


In [9]:
# убираем холодных пользователей из теста

cold_users = set(test[Columns.User]) - set(train[Columns.User])
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)

In [10]:
# заполняем пустые значения

user_df.fillna('Unknown', inplace=True)
user_df = user_df.loc[user_df[Columns.User].isin(train[Columns.User])].copy()
user_df.head()

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0
5,1037719,age_45_54,income_60_90,М,0


In [11]:
# извлекаем возраст, пол и доход пользователей

user_features_frames = []
for feature in ["age", "sex", "income"]:
    feature_frame = user_df.reindex(columns=[Columns.User, feature])
    feature_frame.columns = ["id", "value"]
    feature_frame["feature"] = feature
    user_features_frames.append(feature_frame)
user_features = pd.concat(user_features_frames)
user_features.head()

,id,value,feature
0,973171,age_25_34,age
1,962099,age_18_24,age
3,721985,age_45_54,age
4,704055,age_35_44,age
5,1037719,age_45_54,age


In [12]:
item_df = item_df.loc[item_df[Columns.Item].isin(train[Columns.Item])].copy()
item_df.head()

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."


In [13]:
# извлекаем жанры для каждого фильма и сериала

item_df["genre"] = item_df["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
genre_feature = item_df[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"
genre_feature.head()

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre


In [14]:
# извлекаем тип контента для каждого айтема

content_feature = item_df.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"
content_feature.head()

,id,value,feature
0,10711,film,content_type
1,2508,film,content_type
2,10716,film,content_type
3,7868,film,content_type
4,16268,film,content_type


In [15]:
# извлекаем квантиль года выпуска для каждого айтема

item_df['binned_r_year'] = pd.qcut(item_df['release_year'], q=10, labels=list(range(10)))
release_year_feature = item_df.reindex(columns=[Columns.Item, "binned_r_year"])
release_year_feature.columns = ["id", "value"]
release_year_feature["feature"] = "binned_r_year"
release_year_feature.head()

,id,value,feature
0,10711,1,binned_r_year
1,2508,4,binned_r_year
2,10716,3,binned_r_year
3,7868,5,binned_r_year
4,16268,0,binned_r_year


In [16]:
# извлекаем местоположение студии для каждого айтема

item_df["country"] = item_df["countries"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
country_feature = item_df[["item_id", "country"]].explode("country")
country_feature.columns = ["id", "value"]
country_feature["feature"] = "country"
country_feature.head()

,id,value,feature
0,10711,испания,country
1,2508,сша,country
2,10716,канада,country
3,7868,великобритания,country
4,16268,ссср,country


In [17]:
# конкатенируем таблицы с фичами и проверяем по рандомному id

item_features = pd.concat((genre_feature, content_feature, country_feature, release_year_feature))
item_features[item_features['id'] == 222]

,id,value,feature
4326,222,драмы,genre
4326,222,русские,genre
4326,222,film,content_type
4326,222,ссср,country
4326,222,0,binned_r_year


In [18]:

TEST_USERS = test[Columns.User].unique()


## Аппроксимационный поиск ближайших соседей c помощью Hierarchical Navigable Small World (NMSLIB)

In [19]:
# создаем новый датасет на основе interactions.csv

DATASET = Dataset.construct(
    interactions_df=inter_df,
    user_features_df=user_features,
    cat_user_features=["age", "sex", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", "country", 'binned_r_year', 'content_type'],
)

In [20]:
# обучаем модель с учетом лучших параметров

model = LightFMWrapperModel(
  model=LightFM(
    no_components=32, 
    loss='warp', 
    random_state=100,
    learning_rate=0.05,
    user_alpha=0.1,
    item_alpha=0.1,
  ),
  epochs=2,
  num_threads=1,
)
model.fit(DATASET)

In [21]:
def augment_items(factors):
    normed_factors = np.linalg.norm(factors, axis=1)
    max_norm = normed_factors.max()
    extra_dim = np.sqrt(max_norm ** 2 - normed_factors ** 2).reshape(-1, 1)
    augmented_factors = np.append(factors, extra_dim, axis=1)
    return max_norm, augmented_factors

In [22]:
user_emb, item_emb = model.get_vectors(DATASET)
max_norm, augmented_item_emb = augment_items(item_emb)
extra_zeros = np.zeros((user_emb.shape[0], 1))
augmented_user_emb = np.append(user_emb, extra_zeros, axis=1)

In [23]:
index = nmslib.init(method='hnsw', space='cosinesimil', data_type=nmslib.DataType.DENSE_VECTOR) 
index.addDataPointBatch(augmented_item_emb); 

In [24]:
index_time_params = {'M': 48, 'indexThreadQty': 4, 'efConstruction': 100}
index.createIndex(index_time_params) 
print('Index-time parameters', index_time_params)

Index-time parameters {'M': 48, 'indexThreadQty': 4, 'efConstruction': 100}


In [25]:
query_time_params = {'efSearch': 100}
index.setQueryTimeParams(query_time_params)

In [26]:
def get_map(df, col):
    inv_map = dict(enumerate(df[col].unique()))
    mapp = {v: k for k, v in inv_map.items()}
    return inv_map, mapp

In [27]:
users_inv_map, users_map = get_map(train, 'user_id')
test_emb_ids = [users_map[user] for user in TEST_USERS]
test_emb_ids[:10]
query_matrix = augmented_user_emb[test_emb_ids, :]
query_qty = query_matrix.shape[0]
nbrs = index.knnQueryBatch(query_matrix, k = 10, num_threads = 4)
item_inv_mapping, item_mapping = get_map(train, 'item_id')

In [28]:
# рекомендации тестовым пользователям

recos = [[item_inv_mapping[item] for item in nbrs[i][0]] for i in range(len(nbrs))]
recos = pd.DataFrame(list(zip(TEST_USERS, recos)), columns=['user_id', 'item_id'])
recos.head()

,user_id,item_id
0,203219,"[12846, 2616, 11970, 9617, 7347, 8391, 7737, 3..."
1,200197,"[12846, 2616, 9617, 11970, 7347, 8391, 7737, 3..."
2,73446,"[12846, 2616, 11970, 9617, 7347, 8391, 7737, 3..."
3,10010,"[12846, 2616, 11970, 9617, 7347, 8391, 7737, 3..."
4,890735,"[12846, 2616, 11970, 9617, 7347, 8391, 7737, 3..."


In [29]:
res = recos.explode('item_id')
res['rank'] = res.groupby('user_id').cumcount() + 1
approc_metrics = calc_metrics(METRICS, res, test, train)
approc_metrics

{'recall@10': 0.0028580047340201835, 'map@10': 0.0011749443720157793}

## Решение проблемы холодных пользователей

Проблема холодных пользователей решается многими способами. Например, таким пользователям можно выдавать наиболее популярные запросы. Другой вариант - по имеющимся статистическим данным определять для холодного пользователя наиболее близкую демографическую группу и выдавать наиболее популярные рекомендации для этой группы. Кроме того, можно сделать дефолтный набор продуктов, рекомендуемых новым пользователям. Еще один способ - найти пользователя наиболее близкого к холодному и дать рекомендации как будто бы для него. Этот вариант мы постарались реализовать в коде, представленном ниже.

In [30]:
cold_users = list(cold_users)

In [31]:
cold_users_inv_map, cold_users_map = get_map(inter_df, 'user_id')
citems_inv_map, citems_map = get_map(inter_df, 'item_id')

In [32]:
DATASET = LFMDataset()
DATASET.fit(users=inter_df["user_id"].values,items=inter_df["item_id"].values,)
inter_matrix, _ = DATASET.build_interactions(zip(*inter_df[["user_id", "item_id"]].values.T))

In [33]:
model = LightFM(learning_rate=0.05, 
                loss='warp', 
                no_components=64,
                random_state=100,
)
model.fit(interactions = inter_matrix, 
                        epochs=2,
                        num_threads=20,
)

In [34]:
def get_similar_user(model, user_id):
    user_embeddings = (model.user_embeddings.T
                      / np.linalg.norm(model.user_embeddings, axis=1)).T
    query_embedding = user_embeddings[user_id]
    similarity = np.dot(user_embeddings, query_embedding)
    most_similar = np.argsort(-similarity)[1:11]
    return most_similar

In [35]:
get_similar_user(model, cold_users[100])

array([ 51917, 152300, 272977, 144096, 332726, 157477, 568259, 701735,
       249970, 150317])

In [36]:
id_to_item = {v: k for k, v in DATASET._item_id_mapping.items()}

In [37]:
def get_recos(user_id, model, inter_matrix, user_to_id, id_to_item, n_recommendations = N):
    inner_id = user_to_id[user_id]
    scores = model.predict(user_ids=inner_id,
                            item_ids=np.arange(inter_matrix.shape[1]),
                            num_threads=20)
    watched_items = inter_matrix.col[inter_matrix.row == inner_id]
    scores[watched_items] = -np.inf
    reco_item_inner_ids = np.argpartition(scores, -np.arange(n_recommendations))[-n_recommendations:][::-1]
    recos = [id_to_item[x] for x in reco_item_inner_ids]
    return recos

In [38]:
user_ids = get_similar_user(model, cold_users[0])
user_ids = [idx for idx in user_ids if idx not in cold_users]
recos = get_recos(user_id=user_ids[0],
          model=model,
          inter_matrix=inter_matrix,
          user_to_id=DATASET._user_id_mapping,
          id_to_item=id_to_item,
          n_recommendations=10
)
cold_22_recos = pd.DataFrame({"user_id": user_ids[0], "item_id": recos}).merge(item_df[["item_id", "title", "genres"]])
cold_22_recos

,user_id,item_id,title,genres
0,463081,10440,Хрустальный,"триллеры, детективы"
1,463081,15297,Клиника счастья,"драмы, мелодрамы"
2,463081,13865,Девятаев,"драмы, военные, приключения"
3,463081,9728,Гнев человеческий,"боевики, триллеры"
4,463081,4151,Секреты семейной жизни,комедии
5,463081,3734,Прабабушка легкого поведения,комедии
6,463081,2657,Подслушано,"драмы, триллеры"
7,463081,4880,Афера,комедии
8,463081,6809,Дуров,документальное
9,463081,142,Маша,"драмы, триллеры"
